In [1]:
[(6, 8), (13, 14), (6, 7), (4, 13), (3, 5), (5, 7), (12, 14), (10, 14), (10, 11), (7, 9), (12, 15), (8, 12), (1, 6), (2, 6), (0, 2)]
-21866.90682238832

-21866.90682238832

In [52]:
import numpy as np
import copy
import networkx as nx

def loadfile(filename1, filename2=None):
    ds1 = np.loadtxt(filename1, delimiter=",", dtype=int)
    if filename2:
        ds2 = np.loadtxt(filename1, delimiter=",", dtype=int)
        ds = np.vstack((ds1, ds2))
    else:
        ds = ds1
    return ds, ds.shape[0], ds.shape[1]

def prob_matrix(ds, m, n):
    prob_xy = np.zeros((n, n, 4))
    for i in range(n):
        subds = ds[ds[:, i] == 0]
        for j in range(n):
            if prob_xy[i, j, 0] == 0:
                prob_xy[i, j, 0] = (subds[subds[:, j] == 0].shape[0]+1)/(m+4)
            if prob_xy[j, i, 0] == 0:
                prob_xy[j, i, 0] = prob_xy[i, j, 0]
            if prob_xy[i, j, 1] == 0:
                prob_xy[i, j, 1] = (subds[subds[:, j] == 1].shape[0]+1)/(m+4)
            if prob_xy[j, i, 2] == 0:
                prob_xy[j, i, 2] = prob_xy[i, j, 1]
            
        subds = ds[ds[:, i] == 1]
        for j in range(n):
            if prob_xy[i, j, 2] == 0:
                prob_xy[i, j, 2] = (subds[subds[:, j] == 0].shape[0]+1)/(m+4)
            if prob_xy[j, i, 1] == 0:
                prob_xy[j, i, 1] = prob_xy[i, j, 2]
            if prob_xy[i, j, 3] == 0:
                prob_xy[i, j, 3] = (subds[subds[:, j] == 1].shape[0]+1)/(m+4)
            if prob_xy[j, i, 3] == 0:
                prob_xy[j, i, 3] = prob_xy[i, j, 3]
    return prob_xy

def mutual_info(prob_xy, n):
    I_xy = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i < j:
                I_xy[i, j] = prob_xy[i, j, 0]*np.log(prob_xy[i, j, 0]/(prob_xy[i, i, 0]*prob_xy[j, j, 0])) \
                + prob_xy[i, j, 1]*np.log(prob_xy[i, j, 1]/(prob_xy[i, i, 0]*prob_xy[j, j, 3])) \
                + prob_xy[i, j, 2]*np.log(prob_xy[i, j, 2]/(prob_xy[i, i, 3]*prob_xy[j, j, 0])) \
                + prob_xy[i, j, 3]*np.log(prob_xy[i, j, 3]/(prob_xy[i, i, 3]*prob_xy[j, j, 3]))
    return I_xy

def draw_tree(edge_wts, prnt = False):
    print(edge_wts.shape)
    edge_wts_cp = copy.deepcopy(edge_wts)
    edges = [np.unravel_index(np.argmax(edge_wts_cp), edge_wts_cp.shape)]
    visited = [[edges[-1][0],edges[-1][1]]]
    edge_wts_cp[edges[-1]] = 0
    while(len(edges) < edge_wts.shape[0]-1):
        i = j = -1
        edge = np.unravel_index(np.argmax(edge_wts_cp), edge_wts_cp.shape)
        for bag in range(len(visited)):
            if edge[0] in visited[bag]:
                i = bag
            if edge[1] in visited[bag]:
                j = bag
        if i == -1 and j != -1:
            #print("Visited", visited)
            #print("Added", edge)
            edges.append(edge)
            visited[j].append(edge[0])
        elif i != -1 and j == -1:
            #print("Visited", visited)
            #print("Added", edge)
            edges.append(edge)
            visited[i].append(edge[1])
        elif i == -1 and j == -1:
            #print("Visited", visited)
            #print("Added", edge)
            edges.append(edge)
            visited.append([edge[0], edge[1]])
        elif i != -1 and j != -1 and i != j:
            #print("Visited", visited)
            #print("Added", edge)
            edges.append(edge)
            visited[i] += visited[j]
            visited.remove(visited[j])
        elif i == j != -1:
            pass
            #print("Discarded", edge)
        else:
            #pass
            print("Discarded in else", edge)
        edge_wts_cp[edge] = 0
    #print(edges)
    #print(visited)
    
    new_tree = []
    make_tree(edges, new_tree, edges[0][0])
    
    if prnt:
        G = nx.Graph()
        G.add_nodes_from(visited[0])
        G.add_edges_from(new_tree)
        nx.draw_networkx(G, with_labels=True, arrows=True)
    
    return new_tree

def count_matrix(ds, tree, cols):
    count_xy = np.zeros((len(tree), cols))
    for idx, node in enumerate(tree):
        i, j = node
        count_xy[idx] = [ds[(ds[:, i]==0) & (ds[:, j]==0)].shape[0], ds[(ds[:, i]==0) & (ds[:, j]==1)].shape[0], ds[(ds[:, i]==1) & (ds[:, j]==0)].shape[0], ds[(ds[:, i]==1) & (ds[:, j]==1)].shape[0]]
    #print(count_xy)
    return count_xy

def make_tree(ls, new_tree, parent):
    for node in [item for item in ls if parent in item]:
        if node[0] == parent:
            new_tree.append(node)
            ls.remove(node)
            #print(node, ls, new_tree)
            make_tree(ls, new_tree, node[1])
        else:
            new_tree.append((node[1],node[0]))
            ls.remove(node)
            #print(node, ls, new_tree)
            make_tree(ls, new_tree, node[0])
            
if __name__ == "__main__":
    ds, m, n = loadfile("small-10-datasets/dna.ts.data", "small-10-datasets/dna.valid.data")   
    print(m, n)
    #ds, m, n = loadfile("small-10-datasets/nltcs.ts.data")
    prob_xy = prob_matrix(ds, m, n)
    I_xy = mutual_info(prob_xy, n)
    tree = draw_tree(I_xy, False)
    tree = [(tree[0][0], tree[0][0])] + tree
    print(tree)
    cond_prob = np.zeros((len(tree), prob_xy.shape[2]))
    for idx, node in enumerate(tree):
        if node[0] == node[1]:
            cond_prob[idx] = np.log(prob_xy[node[0], node[1],:])
        else:
            cond_prob[idx] = np.log(np.hstack(((prob_xy[node[0], node[1],:2]/prob_xy[node[0], node[0], 0]),(prob_xy[node[0], node[1],2:]/prob_xy[node[0], node[0], 3]))))
    ts = np.loadtxt("small-10-datasets/dna.test.data", delimiter=",", dtype=int)
    count_xy = count_matrix(ts, tree, prob_xy.shape[2])
    print(np.sum(count_xy*cond_prob)/ts.shape[0])
    #print(edge_wts[np.unravel_index(np.argmax(edge_wts), edge_wts.shape)] == edge_wts.max())

3200 180
(180, 180)
[(87, 87), (87, 89), (89, 88), (88, 92), (92, 90), (92, 91), (92, 104), (104, 103), (103, 107), (107, 106), (106, 105), (106, 110), (110, 108), (110, 109), (109, 113), (113, 112), (112, 111), (112, 116), (116, 115), (115, 114), (115, 119), (119, 118), (118, 122), (122, 121), (121, 125), (125, 124), (124, 128), (128, 127), (127, 131), (131, 130), (130, 129), (130, 134), (134, 133), (133, 132), (133, 137), (137, 136), (136, 140), (140, 138), (140, 139), (139, 143), (143, 142), (142, 146), (146, 145), (145, 149), (149, 148), (148, 152), (152, 151), (151, 155), (155, 153), (155, 154), (154, 158), (158, 157), (157, 161), (161, 160), (160, 164), (164, 163), (163, 162), (163, 167), (167, 166), (166, 170), (170, 169), (169, 173), (173, 172), (172, 171), (172, 176), (176, 175), (175, 174), (175, 179), (179, 178), (179, 177), (170, 168), (167, 165), (161, 159), (158, 156), (152, 150), (149, 147), (146, 144), (143, 141), (137, 135), (128, 126), (125, 123), (122, 120), (119, 11